# Iterative Code generation and Refinement - Development Notebook

This notebook will provide space to develop and test portions of the iterative code generator and refinement, using Llama to generate code in a given language and refine it based on programming problems such as project euler or other available problem sets.

```json
{
  "text": "Write a python function to identify non-prime numbers.", 
  "code": "import math\r\ndef is_not_prime(n):\r\n    result = False\r\n    for i in range(2,int(math.sqrt(n)) + 1):\r\n        if n % i == 0:\r\n            result = True\r\n    return result", 
  "task_id": 3, 
  "test_setup_code": "", 
  "test_list": ["assert is_not_prime(2) == False", "assert is_not_prime(10) == True", "assert is_not_prime(35) == True"], 
  "challenge_test_list": []
}
```

## Globals

In [27]:
import subprocess, json, os, sys, shutil, re, requests
from pathlib import Path


PROBLEM_SET_LOCATION = "/Users/nathanjay/Documents/University Documents/2025 1 Spring/CS6375 Machine Learning/project/semantic-parser/early_development/problems.jsonl"

DEFAULT_PROMPT = """
You are a python expert. You will be given a prompt to create a python funcion.
Your task is to create a python function that solves the problem described in the prompt.

**IMPORTANT**
- Your response must be ONLY valid JSON with no additional text, markdown formatting, backticks, or explanations before or after the JSON. Do not wrap your response in ```json or ``` markers.
- The JSON object should start with {{ and end with }} with nothing else before or after.
- The JSON object should contain the following fields in this order:
  - `explanation`: a short explanation of the code and your approach.
  - `code`: the code of the function.
- In valid JSON, both keys and values should be in double quotes. (e.g., "key": "value").


**EXPLANATION RULES**
- The explanation should be no more than 3 sentences.
- The explanation should be in plain text with no formatting.
- The explanation should describe the code, the approach, and any important details.

**CODE RULES**
- The function should be a valid python function.
- The function should be named `candidate`.
- The function should not contain any print statements.
- The function should contain explanatory comments.
- Only output a single function. If you are unsure, output the most universal version of the function.
- If subfunctions are needed, they should be defined inside the main function.
- The `code` block in the JSON object should include newlines and indentation.
"""

HOSTNAME = "localhost"
PORT = 11434
OLLAMA_API_URL = f"http://{HOSTNAME}:{PORT}/api/generate"
MODEL = "llama3.2:1b"

In [ ]:
"""
You are a python expert. You will be given a prompt to create a python funcion.
Your task is to create a python function that solves the problem described in the prompt.

**IMPORTANT**
- Your response must be ONLY valid JSON with no additional text, markdown formatting, backticks, or explanations before or after the JSON. Do not wrap your response in ```json or ``` markers.
- The JSON object should start with { and end with } with nothing else before or after.
- The JSON object should contain the following fields in this order:
  - `explanation`: a short explanation of the code and your approach.
  - `code`: the code of the function.
  - `test_cases`: a list of test cases for the function.
- In valid JSON, both keys and values should be in double quotes. (e.g., "key": "value").


**CODE RULES**
- The function should be a valid python function.
- The function should be named `candidate`.
- The function should not contain any print statements.
- The function should contain explanatory comments.
- Only output a single function. If you are unsure, output the most universal version of the function.
- If subfunctions are needed, they should be defined inside the main function.
- The `code` block in the JSON object should include newlines and indentation.

**TEST CASES RULES**
- The test cases should be a list of dictionaries.
- Each dictionary should contain the following fields:
  - `input`: the input to the function.
  - `expected_output`: the expected output of the function.
- The input should be a valid input for the function. 
- If the function takes no input, the input should be an empty dictionary, and only one test case should be provided.
"""

## Step 1: Read Problem set and prompt AI

In [2]:
def load_problem_set():
  """
  Load the problem set from the JSONL file.
  Each line in the file is a separate JSON object.
  """
  problems = []
  with open(PROBLEM_SET_LOCATION, "r") as f:
    for line in f:
      if line.strip():  # Skip empty lines
        problems.append(json.loads(line))
  return problems

def get_problem_description(problem):
  """
  Get the problem description from the problem dictionary.
  The description is in the 'text' field.
  """
  return problem.get("text", "")

def get_code_prompt(problem):
  """
  Create a prompt for the code generation model.
  The prompt includes the problem description and the default prompt.
  """
  problem_description = get_problem_description(problem)
  return f"{DEFAULT_PROMPT}\n\n{problem_description}\n\n"

def generate_code_with_ollama(prompt, model=MODEL, temperature=0.7):
    """
    Generate code using Ollama API
    
    Args:
        prompt (str): The prompt to send to Ollama
        model (str): The model to use (default: llama3)
        temperature (float): Controls randomness (0.0-1.0)
        
    Returns:
        str: Generated code
    """
    url = OLLAMA_API_URL
    
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "temperature": temperature
    }
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        result = response.json()
        return result["response"]
    except requests.exceptions.RequestException as e:
        print(f"Error calling Ollama API: {e}")
        return None
    
def solve_problem_with_ollama(problem, model=MODEL):
    """
    Generate code for a given problem using Ollama
    
    Args:
        problem (dict): Problem dictionary with text description
        model (str): Ollama model to use
        
    Returns:
        str: Generated code solution
    """
    prompt = get_code_prompt(problem)
    solution = generate_code_with_ollama(prompt, model)
    return solution

In [35]:
# main
problems = load_problem_set()

sample_problem = problems[0]
solution = solve_problem_with_ollama(sample_problem)
print(f"Problem: {get_problem_description(sample_problem)}")
print("\nGenerated Solution:")
print(solution)

Problem: Write a python function to always return 1.

Generated Solution:
{"explanation": "This function is named `candidate` because it returns the single constant value of 1. It does not contain any print statements and only uses basic Python operations.", "code": "def candidate():\n    # This comment explains that we don't need any output, so no print statements are needed\n    # We simply return a universal constant value when no other function is called with it", "candidate": 1}


In [ ]:
def extract_json_from_response(response):
  """
  Extract JSON from the response string.
  
  Args:
    response (str): The response string from the model.
      
  Returns:
    dict: The extracted JSON object.
  """


  # Try to find JSON content between curly braces
  json_match = re.search(r'(\{.*\})', response, re.DOTALL)
  if json_match:
    json_str = json_match.group(1)
    print(f"Extracted JSON string: {json_str}")
    try:
      return json.loads(json_str)
    except json.JSONDecodeError:
      pass

  # If no valid JSON found with regex, try other approaches
  try:
    # Remove code block markers if present
    cleaned = re.sub(r'```json|```', '', response).strip()
    return json.loads(cleaned)
  except json.JSONDecodeError:
    print("Could not extract valid JSON from response")
    return None

In [36]:
# extracted_json = extract_json_from_response("{"+solution)
extracted_json = extract_json_from_response(solution)

print(json.dumps(extracted_json, indent=2))

Extracted JSON string: {"explanation": "This function is named `candidate` because it returns the single constant value of 1. It does not contain any print statements and only uses basic Python operations.", "code": "def candidate():\n    # This comment explains that we don't need any output, so no print statements are needed\n    # We simply return a universal constant value when no other function is called with it", "candidate": 1}
{
  "explanation": "This function is named `candidate` because it returns the single constant value of 1. It does not contain any print statements and only uses basic Python operations.",
  "code": "def candidate():\n    # This comment explains that we don't need any output, so no print statements are needed\n    # We simply return a universal constant value when no other function is called with it",
  "candidate": 1
}


In [ ]:
def candidate():
    # This comment explains that we don't need any output, so no print statements are needed
    # We simply return a universal constant value when no other function is called with it



def test_candidate():
  """
    A simple test function to check the output of the candidate function.
    
    This function calls the candidate function and prints its output.
    """
  # Call the candidate function and print its output
  result = candidate()
  print(f"Candidate function output: {result}")

test_candidate()

Candidate function output: 1
